In [1]:
import os
import shutil

origin = "/mnt/c/Users/hajun/Project/image_1/dataset"
base = "/mnt/c/Users/hajun/Project/image_1/splitted"
clss_list = os.listdir(origin)
os.mkdir(base)

train_dir = os.path.join(base, "train")
val_dir = os.path.join(base, "val")
test_dir = os.path.join(base, "test")

os.mkdir(train_dir)
os.mkdir(val_dir)
os.mkdir(test_dir)

for clss in clss_list:
  os.mkdir(os.path.join(train_dir, clss))
  os.mkdir(os.path.join(val_dir, clss))
  os.mkdir(os.path.join(test_dir, clss))

In [2]:
import math

for clss in clss_list:
  path = os.path.join(origin, clss)
  fnames = os.listdir(path)

  train_size = math.floor(len(fnames) * 0.6)
  val_size = math.floor(len(fnames) * 0.2)
  test_size = math.floor(len(fnames) * 0.2)

  train_fnames = fnames[:train_size]
  for fname in train_fnames:
    src = os.path.join(path, fname)
    dst = os.path.join(os.path.join(train_dir, clss), fname)
    shutil.copyfile(src, dst)
    
  val_fnames = fnames[train_size : (train_size + val_size)]
  for fname in val_fnames:
    src = os.path.join(path, fname)
    dst = os.path.join(os.path.join(val_dir, clss), fname)
    shutil.copyfile(src, dst)
  
  test_fnames = fnames[(train_size + val_size) : (train_size + val_size + test_size)]
  for fname in test_fnames:
    src = os.path.join(path, fname)
    dst = os.path.join(os.path.join(test_dir, clss), fname)
    shutil.copyfile(src, dst)

In [3]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

use_cuda = torch.cuda.is_available()
print(use_cuda)
device = torch.device("cuda" if use_cuda == 1 else "cpu") # 연산에 사용할 device 정의

path = "/mnt/c/Users/hajun/Project/image_1"
epoch = 10 
batch_size = 256

transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()]) 
# 이미지 전처리, Augmentation등에 활용, 위 코드는 이미지 크기를 (64, 64)로 조정, 이미지 데이터를 tensor로 변환

train_dataset = ImageFolder(root = path + "/splitted/train", transform = transform_base)
val_dataset = ImageFolder(root = path + "/splitted/val", transform = transform_base)
# 하나의 폴더가 하나의 클래스에 대응하는 구조를 가진 폴더+파일에 사용

from torch.utils.data import DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = True, num_workers = 4)
# 불러온 이미지 데이터를 조건에 맞게 미니배치 단위로 분리하는 역할

True


In [4]:
# 베이스라인 모델 설계

import torch.nn as nn    # nn.Module에 딥러닝과 관련된 기본적인 함수들 포함
import torch.nn.functional as F  
import torch.optim as optim  # 최적화 옵션

class Net(nn.Module):  # nn.Module 상속받음 -> 다 끌어와 씀
  def __init__(self):
    super(Net, self).__init__()
    
    self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
    self.conv3 = nn.Conv2d(64, 64, 3, padding = 1)
    # padding=1에 커널 크기3으로 층 지나도 이미지 크기 보존
    # Conv2d의 앞 세 개 파라미터 -> 입력 채널 수, 출력 채널 수, 필터사이즈
    # 따라서 위 세 개의 합성곱연산을 지나면 채널이 3->32->64->64로, 이미지 크기는 그대로

    self.fc1 = nn.Linear(4096, 512) # 합성곱 연산에서 이미지 크기 64*64 --flatten --> 4096을 512로 출력
    self.fc2 = nn.Linear(512, 33)  # 512받고  33 -> class 개수로 출력

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p = 0.25, training = self.training) # 25%의 노드를 dropout, training -> 학습과정에서만 dropout적용

    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p = 0.25, training = self.training) 

    x = self.conv3(x)
    x = F.relu(x)
    x = self.pool(x)
    x = F.dropout(x, p = 0.25, training = self.training) 

    x = x.view(-1, 4096) # numpy의 reshape
    x = self.fc1(x)
    x = F.relu(x)
    x = F.dropout(x, p = 0.5, training = self.training) 
    x = self.fc2(x)

    return F.log_softmax(x, dim = 1)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001) # 최적화 옵션으로 Adam사용

In [5]:
# 모델 학습을 위한 함수
def train(model, train_loader, optimizer):  # 모델 학습에 필요한 것 -> 모델, 데이터, 최적화옵셥
  model.train() #모델을 학습모드로 변경
  for batch_idx, (data, target) in enumerate(train_loader):  # train_loader는 조건에 맞게 미니배치 단위로 나뉘어져있어 여기 for문처럼 변수할당
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad() # for문이 배치 단위 -> 이전 배치 Gradient값(가중치 미분값) 초기화
    output = model(data)  # 모델에 input넣고 output뽑기
    loss = F.cross_entropy(output, target)  # loss값 뽑기
    loss.backward()  # 오차역전파
    optimizer.step() # 역전파한 값들로 모델 파라미터 업데이트

# 모델 평가 함수

def evaluate(model, test_loader):
  model.eval() # 모델 평가모드로 변경
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)

      test_loss += F.cross_entropy(output, target, reduction = "sum").item()  # loss값은 교차엔트로피 총합으로! -> default는 mean
      pred = output.max(1, keepdim = True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(test_loader.dataset) # 이전 test_loss는 배치별 loss총합이 더해진 값. 이를 배치 개수로 나누어 평균 계산
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

In [6]:
import time
import copy
def train_baseline(model, train_loader, val_loader, optimizer, num_epochs = 30):
  best_acc = 0.0
  best_model_wts = copy.deepcopy(model.state_dict()) # 정확도 가장 높은 모델 저장

  for epoch in range(1, num_epochs+1):
    since = time.time()
    train(model, train_loader, optimizer)
    train_loss, train_acc = evaluate(model, train_loader)
    val_loss, val_acc = evaluate(model, val_loader)

    if val_acc > best_acc:
      best_acc = val_acc
      best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f"------------------------------------- epoch {epoch} -------------------------------------")
    print("train Loss : {:.4f}, Accuracy : {:.2f}%".format(train_loss, train_acc))
    print("val_Loss : {:.4f}, Accuracy : {:.2f}%".format(val_loss, val_acc))
    print("Completed in {:.0f}m {:.0f}s".format(time_elapsed//60, time_elapsed % 60))
  model.load_state_dict(best_model_wts) # 가장 정확도 높은 모델 반환
  return model

base = train_baseline(model, train_loader, val_loader, optimizer, epoch)
torch.save(base, "baseline.pt")

RuntimeError: shape '[-1, 4096]' is invalid for input of size 44282880

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

batch_size = 256
epoch = 30

data_transforms = {
    "train" : transforms.Compose([
        transforms.Resize([64, 64]),  
        transforms.RandomHorizontalFlip(),  # base model과 다르게 augmentation을 적용하는데, 
        transforms.RandomVerticalFlip(),    # pre_trained model이라 데이터 양을 더 늘린 것 같음(자료에 이유가 안 나와있다..) 
        transforms.RandomCrop(52),  # 이미지 일부를 52*52로 잘라내어씀
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],  # 정규화에 쓰일 각 채널의 평균값
                            [0.229, 0.224, 0.225])   # 정규화에 쓰일 각 채널의 표준편차값
    ]),
    "val" : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
    ]),
    
}

data_dir = path + "/splitted"

dataset = {x : ImageFolder(root = os.path.join(data_dir, x), transform = data_transforms[x]) for x in ["train", "val"]}
loader = {x : torch.utils.data.DataLoader(dataset[x], batch_size = batch_size, shuffle = True, num_workers = 4) for x in ["train", "val"]}
dataset_sizes = {x : len(dataset[x]) for x in ["train", "val"]}
class_names = dataset["train"].classes

In [ ]:
from torchvision import models
resnet = models.resnet50(pretrained = True) # False가 되면 모델의 구조만 가져오고 초깃값은 랜덤 설정
num_ftrs = resnet.fc.in_features # fc는 모델의 마지막 layer를, in_features는 해당 층의 입력 채널 수 반환
resnet.fc = nn.Linear(num_ftrs, 33) # 마지막 fc층의 출력 채널을 클래스 수에 맞게 변환
resnet = resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p : p.requires_grad, resnet.parameters()), lr=0.001)
# filter와 lambda를 통해 requires_grad=True인 layer만 파라미터 업데이트

from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)
# 에폭에 따라 lr변경 -> 7에폭마다 0.1씩 곱해짐

/home/hajun/anaconda3/envs/image/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hajun/anaconda3/envs/image/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/hajun/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%


In [ ]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs = 25):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0
    
    for epoch in range(num_epochs):                           # 에폭마다 for문
        print(f"---------- epoch {epoch + 1} ----------")
        since = time.time()
        
        for phase in ["train", "val"]:                        # train과 val데이터 동시에-> for문인거 신경 안써도됨
            if phase == "train":
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in loader[phase]:             # 배치단위마다 for문
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    x, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == "train":
                        loss.backward() 
                        optimizer.step()       
                        
                running_loss += loss.item() * inputs.size(0)       # 교차엔트로피 계산 deafualt값이 mean이므로 각 데이터 마다의 손실 평균이 저장되있음
                                                                   # 따라서 배치 사이즈를 곱해줘 한 배치 사이즈의 loss 총합을 계산!
                running_corrects += torch.sum(preds == labels.data)  # -----------------여기까진 base model과 구조가 동일----------------------------
            if phase == "train":
                scheduler.step()
                l_r = [x["lr"] for x in optimizer_ft.param_groups]
                print("learning rate : ", l_r)
                 
            epoch_loss = running_loss/dataset_sizes[phase]          # 전체 데이터 loss합을 각 데이터셋 전체 크기로 나눠주어 loss계산
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print("Completed in {:.0f}m {:0f}s".format(time_elapsed // 60, time_elapsed % 60))
        
    print("Best val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model

In [ ]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=epoch)
torch.save(model_resnet50, "resnet50.pt")

---------- epoch 1 ----------
learning rate :  [0.001]
train Loss: 3.354830 Acc: 0.0611
val Loss: 2.470476 Acc: 0.4500
Completed in 0m 4.852816s
---------- epoch 2 ----------
learning rate :  [0.001]
train Loss: 1.296646 Acc: 0.7278
val Loss: 3.960766 Acc: 0.4500
Completed in 0m 4.685889s
---------- epoch 3 ----------
learning rate :  [0.001]
train Loss: 0.521374 Acc: 0.8333
val Loss: 2.866120 Acc: 0.5500
Completed in 0m 4.757969s
---------- epoch 4 ----------
learning rate :  [0.001]
train Loss: 0.430191 Acc: 0.8611
val Loss: 0.792002 Acc: 0.8167
Completed in 0m 4.926021s
---------- epoch 5 ----------
learning rate :  [0.001]
train Loss: 0.337188 Acc: 0.8611
val Loss: 1.604713 Acc: 0.8000
Completed in 0m 4.419656s
---------- epoch 6 ----------
learning rate :  [0.001]
train Loss: 0.267165 Acc: 0.9111
val Loss: 0.606081 Acc: 0.8833
Completed in 0m 5.123018s
---------- epoch 7 ----------
learning rate :  [0.0001]
train Loss: 0.139732 Acc: 0.9333
val Loss: 1.832132 Acc: 0.7833
Completed 